In [1]:
import cv2

In [2]:
from ultralytics import YOLOv10

c:\Users\User\anaconda3\envs\Headcount\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import cv2
import numpy as np
import torch

torch.cuda.is_available()

True

In [4]:
# model = YOLOv10("runs/detect/train40/weights/best.pt")
model = YOLOv10("weights/best.pt")

c:\Users\User\Desktop\yolov10-main\ultralytics\nn\tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


In [5]:
# # 비디오 파일 경로 설정
# video_path = 'test003.mp4'
# cap = cv2.VideoCapture(video_path)

# # 비디오 저장 설정 (옵션)
# output_path = 'output_video.mp4'
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# fps = int(cap.get(cv2.CAP_PROP_FPS))
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # 프레임에서 객체 탐지
#     results = model(
#         frame,
#         conf=0.1,
#         device='cuda'
#     )

#     # 탐지 결과 처리 (boxes 사용)
#     boxes = results[0].boxes  # 탐지된 객체의 바운딩 박스 정보

#     for box in boxes:
#         x1, y1, x2, y2 = map(int, box.xyxy[0])  # 바운딩 박스 좌표
#         conf = box.conf[0]  # 신뢰도
#         cls = int(box.cls[0])  # 클래스 레이블
        
#         # 객체 상자 그리기
#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#         # 라벨과 신뢰도 텍스트 그리기
#         cv2.putText(frame, f'Class {cls} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

#     # 출력 영상 저장 (옵션)
#     out.write(frame)

#     # 화면에 결과 프레임 표시 (옵션)
#     cv2.imshow('YOLOv10 Video Detection', frame)

#     # 'q' 키를 눌러 중지
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # 리소스 해제
# cap.release()
# out.release()
# cv2.destroyAllWindows()

In [6]:
# /

In [10]:
# 비디오 파일 경로 설정
video_path = 'test004.mp4'
cap = cv2.VideoCapture(video_path)

# 비디오 저장 설정 (옵션)
output_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 배치 처리 크기
batch_size = 48  # 한 번에 처리할 프레임 수 (GPU 메모리에 따라 변경 가능)
frames_batch = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임을 배치에 추가
    frames_batch.append(frame)

    # 배치가 채워지면 한 번에 추론
    if len(frames_batch) == batch_size:
        # 배치에 있는 모든 프레임을 YOLOv10 입력 크기로 리사이즈 및 처리
        input_batch = [cv2.resize(f, (640, 640)) for f in frames_batch]

        # 입력 배치를 텐서로 변환하고 GPU로 이동
        input_tensor = torch.from_numpy(np.stack(input_batch))\
            .permute(0, 3, 1, 2).float().cuda() / 255.0

        # 배치로 추론 수행
        with torch.no_grad():
            results = model(
                input_tensor,
                conf = 0.01,
                # max_det = 300,
                # iou=0.9,
                # augment=True,
                # retina_masks = True,
                # agnostic_nms = True,
                device='cuda'
            )

        # 결과를 원본 프레임에 맞춰 시각화
        for i, result in enumerate(results):
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0]  # 신뢰도
                cls = int(box.cls[0])  # 클래스 레이블

                # 원본 프레임으로 좌표 복원
                x1 = int(x1 * (width / 640))
                x2 = int(x2 * (width / 640))
                y1 = int(y1 * (height / 640))
                y2 = int(y2 * (height / 640))

                # 객체 상자 그리기
                cv2.rectangle(frames_batch[i], (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frames_batch[i], f'{conf:.2f}', (x1, y1 - 10),
                             cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 0), 2)

        # 출력 영상 저장
        for frame in frames_batch:
            out.write(frame)
            # 화면에 결과 프레임 표시 (옵션)
#            cv2.imshow('YOLOv10 Video Detection', frame)

        # 'q' 키를 눌러 중지
#        if cv2.waitKey(1) & 0xFF == ord('q'):
 #           break
        

        # 배치 초기화
        frames_batch = []

# 리소스 해제
cap.release()
out.release()
cv2.destroyAllWindows()


0: 640x640 89 heads, 19.4ms
1: 640x640 92 heads, 19.4ms
2: 640x640 89 heads, 19.4ms
3: 640x640 84 heads, 19.4ms
4: 640x640 116 heads, 19.4ms
5: 640x640 104 heads, 19.4ms
6: 640x640 113 heads, 19.4ms
7: 640x640 98 heads, 19.4ms
8: 640x640 100 heads, 19.4ms
9: 640x640 104 heads, 19.4ms
10: 640x640 106 heads, 19.4ms
11: 640x640 97 heads, 19.4ms
12: 640x640 82 heads, 19.4ms
13: 640x640 94 heads, 19.4ms
14: 640x640 108 heads, 19.4ms
15: 640x640 103 heads, 19.4ms
16: 640x640 108 heads, 19.4ms
17: 640x640 106 heads, 19.4ms
18: 640x640 101 heads, 19.4ms
19: 640x640 97 heads, 19.4ms
20: 640x640 98 heads, 19.4ms
21: 640x640 90 heads, 19.4ms
22: 640x640 95 heads, 19.4ms
23: 640x640 103 heads, 19.4ms
24: 640x640 110 heads, 19.4ms
25: 640x640 107 heads, 19.4ms
26: 640x640 110 heads, 19.4ms
27: 640x640 104 heads, 19.4ms
28: 640x640 117 heads, 19.4ms
29: 640x640 101 heads, 19.4ms
30: 640x640 100 heads, 19.4ms
31: 640x640 94 heads, 19.4ms
32: 640x640 102 heads, 19.4ms
33: 640x640 103 heads, 19.4ms
34

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
